<a href="https://colab.research.google.com/github/NavaneethaCSR/Sociability-Level-Prediction/blob/main/ORGTAMPERED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from keras.preprocessing import image
import os
import cv2
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, Add, ZeroPadding2D, AveragePooling2D, Flatten, Dense,Dropout
from tensorflow.keras import layers, models,regularizers
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:

train_data='/content/drive/MyDrive/Colab Notebooks/Datasets/ORGTAMP/train'

validation_data='/content/drive/MyDrive/Colab Notebooks/Datasets/ORGTAMP/validation'
test_data='/content/drive/MyDrive/Colab Notebooks/Datasets/ORGTAMP/test'

## **DATA AUGMENTATION**

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

validation_datagen = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input )
test_datagen = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input )
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Datasets/ORGTAMP/train',
    target_size=(224, 224),
    batch_size=64,

    class_mode='binary',
    seed=32,
    interpolation='bicubic'
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Datasets/ORGTAMP/validation',
    target_size=(224, 224),
    batch_size=64,
    color_mode='grayscale',
    class_mode='binary',
    seed=32,
    interpolation='bicubic'
)
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Datasets/ORGTAMP/test',
    target_size=(224, 224),
    batch_size=64,

    class_mode='binary',
    seed=32,
    interpolation='bicubic'
)

Found 96 images belonging to 2 classes.
Found 24 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


## **DATA PRE-PROCESSING**

In [ ]:
def create_vgg16_model(input_shape=(224, 224, 3), num_classes=2):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers[:-8]:
        layer.trainable = False
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = create_vgg16_model(input_shape=(224, 224, 3), num_classes=1)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

### **TRAINING**

In [ ]:
model.fit(train_generator,validation_data=test_generator,epochs=40,callbacks=[early_stopping])

Epoch 1/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 139s 51s/step - accuracy: 0.6267 - loss: 0.6151 - val_accuracy: 0.6667 - val_loss: 0.6349
Epoch 2/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 144s 54s/step - accuracy: 0.5486 - loss: 0.6462 - val_accuracy: 0.6667 - val_loss: 0.6350
Epoch 3/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 132s 88s/step - accuracy: 0.7083 - loss: 0.5700 - val_accuracy: 0.7500 - val_loss: 0.6182
Epoch 4/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 125s 84s/step - accuracy: 0.6944 - loss: 0.6175 - val_accuracy: 0.5833 - val_loss: 0.6382
Epoch 5/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 142s 45s/step - accuracy: 0.6233 - loss: 0.6182 - val_accuracy: 0.6667 - val_loss: 0.6357
Epoch 6/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 126s 84s/step - accuracy: 0.6562 - loss: 0.5979 - val_accuracy: 0.7500 - val_loss: 0.6409
Epoch 7/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 142s 83s/step - accuracy: 0.7188 - loss: 0.5375 - val_accuracy: 0.6667 - val_loss: 0.6300


## **TESTING**

In [ ]:
model.evaluate(test_generator)

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.7500 - loss: 0.6182


[0.6181960701942444, 0.75]